In [3]:
import pandas as pd
import requests
import os
import time

matches = pd.read_csv('WCMatches.csv')
matches.columns
matches[["winnerid","loserid"]]




,winnerid,loserid
0,7d05pfwn,jfvflnna
1,o8q1tnkp,o8q1tnkp
2,akq5t76h,akq5t76h
3,ng9mpb4s,ng9mpb4s
4,ng9mpb4s,ng9mpb4s
...,...,...
2783,dq44xncy,222vn7x7
2784,g7umra42,g7umra42
2785,g7umra42,222vn7x7
2786,g7umra42,dq44xncy


In [11]:
# Ensure unique_ids is unique
unique_ids = pd.concat([matches['winnerid'], matches['loserid']]).unique()
print(unique_ids.size)
player_info = pd.DataFrame()

1210


In [12]:
import os
# Load existing player_info if it exists, otherwise create empty DataFrame
player_info_path = "EloPlayerinfo.csv"
if os.path.exists(player_info_path):
    player_info = pd.read_csv(player_info_path, low_memory=False)
    processed_ids = set(player_info['player_id'].unique())
else:
    player_info = pd.DataFrame()
    processed_ids = set()

# Only process player_ids that are not already in player_info
to_process = [pid for pid in unique_ids if pid not in processed_ids]

for idx, player_id in enumerate(to_process, start=len(processed_ids)):
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #add player id
    merged_df.insert(loc=0, column= "player_id", value=player_id)

    # Append merged_df to player_info
    player_info = pd.concat([player_info, merged_df], ignore_index=True)

    print(f"Index: {idx}, Player ID: {player_id}")
    time.sleep(0.5) #wait 0.5 second to not pummel the server

# Save player_info after all data is collected
player_info.to_csv("EloPlayerinfo.csv", index=False)
print("List done")

List done


In [74]:
#find if new id's that have not been in the list before have come up
player_info_unique_opps = player_info[["opp_id", "opp_opp"]].drop_duplicates()
matches_unique_players = matches[["winnerid","Target"]].drop_duplicates()
matches_unique_opps = matches[["loserid","Source"]].drop_duplicates()

# Combine all into a single DataFrame with columns: id, name
all_pairs = pd.concat([
    player_info_unique_opps.rename(columns={"opp_id": "id", "opp_opp": "name"}),
    matches_unique_players.rename(columns={"winnerid": "id", "Target": "name"}),
   matches_unique_opps.rename(columns={"loserid": "id", "Source": "name"})
], ignore_index=True)

# Drop duplicates and rows with missing names or ids
all_pairs = all_pairs.dropna(subset=["id", "name"]).drop_duplicates()

# Save the result
all_pairs.to_csv("UniqueIdNamePairs.csv", index=False)


unique_ids_update = pd.DataFrame(all_pairs["id"].unique())
print(unique_ids_update.size)
unique_ids_update.to_csv("UniqueIds.csv", index=False)



90464


In [4]:

import requests
from bs4 import BeautifulSoup
import json
import sys

def get_player_data_from_url(url):
    """
    Fetches HTML from a URL, finds the correct astro-island element,
    and extracts player data from its props attribute.
    """
    try:
        # Fetch the HTML content from the URL
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the astro-island element that corresponds to the Profile component.
        # We'll use the 'component-url' attribute to find it.
        # You might need to adjust the component_url value if it changes.
        # Alternatively, you could use 'component-export': 'default'
        # or look for a unique data attribute if available.
        component_url_to_find = '/_astro/Profile.FIYvQsk3.js'
        astro_island_element = soup.find('astro-island', {'component-url': component_url_to_find})

        # If the above doesn't work or the URL changes, you could try finding
        # based on the `component-export` attribute, which is often 'default':
        # astro_island_element = soup.find('astro-island', {'component-export': 'default'})

        # Or if you need to be very specific, you could find all and iterate
        # checking for a specific player ID within the props:
        # all_astro_islands = soup.find_all('astro-island')
        # for island in all_astro_islands:
        #    props_attribute = island.get('props')
        #    if props_attribute:
        #        try:
        #            props_data = json.loads(props_attribute)
        #            info_data = props_data.get('info', [None, {}])[1]
        #            player_id_in_props = info_data.get('id', [None, None])[1]
        #            if player_id_in_props == target_player_id: # You'd need to pass target_player_id
        #                astro_island_element = island
        #                break
        #        except (json.JSONDecodeError, IndexError, TypeError):
        #             continue # Skip if props can't be parsed or data structure is unexpected


        if astro_island_element:
            props_attribute = astro_island_element.get('props')
            if props_attribute:
                try:
                    # The props attribute value is a JSON string
                    # In the provided data, the structure seems to be a list where the second element
                    # contains the 'info' object.
                    props_data = json.loads(props_attribute)

                    # Accessing the info object. Based on your provided data structure,
                    # the 'info' object is the second element in the main list value
                    # associated with the key "info".
                    info_data = props_data.get('info', [None, {}])[1]

                    player_id = info_data.get('id', [None, None])[1]
                    first_name = info_data.get('first_name', [None, None])[1]
                    last_name = info_data.get('last_name', [None, None])[1]

                    #print(f"ID: {player_id}")
                    #print(f"First Name: {first_name}")
                    #print(f"Last Name: {last_name}")
                    info = [player_id,last_name,first_name]
                    return info
                    

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from props attribute: {e}")
                except Exception as e:
                    print(f"An error occurred while processing data: {e}")
            else:
                print("Error: astro-island element found, but no 'props' attribute.")
        else:
            print(f"Error: Could not find an astro-island element with component-url '{component_url_to_find}'.")
            print("Consider adjusting the selector or trying an alternative method (commented out in the script).")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [ ]:
test_ids = ["l8ugp31t","5ysod3tm","h0x63109","l8re36n5"]
#unique_ids_update.columns

#print(unique_ids_update[unique_ids_update["0"]== ""])


print(unique_ids_update)

              0
0             0
1      f917lein
2      yqvwjvgy
3      jkdsvj4j
4      fodt8dk9
...         ...
90460  f6qnb7rc
90461  cs89l9zw
90462  22aa0u2d
90463  ce94zb1b
90464  mbat0n2a

[90465 rows x 1 columns]


In [8]:
import os
import pandas as pd
import time
import logging

# Set up logging
logging.basicConfig(filename='scrape_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

def load_existing_data(player_list_path, unique_ids_update_path):
    # Load existing UniquePlayerList if it exists, otherwise create an empty DataFrame
    unique_ids_update = pd.read_csv(unique_ids_update_path, header=None)
    if os.path.exists(player_list_path):
        unique_player_list = pd.read_csv(player_list_path)
        processed_ids = set(unique_player_list['ID'].unique())
    else:
        unique_player_list = pd.DataFrame()
        processed_ids = set()
    
    return unique_ids_update, processed_ids

def save_to_csv(player_list_path, player_data_list, header=False):
    # Save data to CSV in bulk
    if player_data_list:
        player_df = pd.DataFrame(player_data_list, columns=["ID", "LastName", "FirstName"])
        player_df.to_csv(player_list_path, mode='a', header=header, index=False)

def process_players(unique_ids_update, processed_ids, player_list_path, save_interval=100):
    player_list = []
    to_process = [pid for pid in unique_ids_update[0] if pid not in processed_ids]
    
    # Keep track of whether the header has been written
    header_written = False

    for idx, id in enumerate(to_process, start=len(processed_ids)):
        try:
            url_player = f"https://mtgeloproject.net/profile/{id}"
            player_data = get_player_data_from_url(url_player)
            player_list.append(player_data)
            print(f"Index: {idx}, Player ID: {id}, Data: {player_data}")
            
            # Save periodically to CSV
            if (idx + 1) % save_interval == 0:
                save_to_csv(player_list_path, player_list, header=not header_written)
                logging.info(f"Saved {len(player_list)} players after processing Player ID {id}")
                player_list = []  # Clear the buffer after saving
                header_written = True  # Mark the header as written

            time.sleep(0.1)  # Throttle requests

        except Exception as e:
            # Log the player ID that caused the error
            logging.error(f"Error processing Player ID {id}: {e}")
            print(f"Error processing Player ID {id}: {e}")
            continue  # Skip to the next player ID on error

        # Log the successful processing of an ID
        logging.info(f"Successfully processed Player ID {id}")
    
    # Save any remaining data after the loop finishes
    if player_list:
        save_to_csv(player_list_path, player_list, header=not header_written)
        logging.info(f"Final save with {len(player_list)} remaining players.")

##Execute the main processing functions
player_list_path = "UniquePlayerList.csv"
unique_ids_update_path = "UniqueIds.csv"
# Load existing data
unique_ids_update, processed_ids = load_existing_data(player_list_path, unique_ids_update_path)
# Process players and save data periodically
process_players(unique_ids_update, processed_ids, player_list_path, save_interval=100)
print("Processing complete.")




Index: 10, Player ID: 0, Data: [None, None, None]
Index: 11, Player ID: i2kzizct, Data: ['i2kzizct', 'Matarese', 'Chris']
Index: 12, Player ID: sannwwjw, Data: ['sannwwjw', 'Schraut', 'Andreas']
Index: 13, Player ID: h6ke3hcc, Data: ['h6ke3hcc', 'Wolf', 'Andrew']
Index: 14, Player ID: r99or99j, Data: ['r99or99j', 'Ladra', 'Lukas']
Index: 15, Player ID: lewcamij, Data: ['lewcamij', 'Ogilbee', 'Brian']
Index: 16, Player ID: d7k7077v, Data: ['d7k7077v', 'Glodt', 'Romain']
Index: 17, Player ID: me4nlw5o, Data: ['me4nlw5o', 'Guevin', 'Thomas']
Index: 18, Player ID: pzh1wbfb, Data: ['pzh1wbfb', 'Moss-Allison', 'Nicolas']
Index: 19, Player ID: 6mia5w8m, Data: ['6mia5w8m', 'Konstanczer', 'Andre']
Index: 20, Player ID: 897497lk, Data: ['897497lk', 'Chinnock', 'John']
Index: 21, Player ID: gpnoyxve, Data: ['gpnoyxve', 'Eskeland', 'Sigurd']
Index: 22, Player ID: lvzgd7gn, Data: ['lvzgd7gn', 'Eikefet', 'Kim']
Index: 23, Player ID: z2315xnw, Data: ['z2315xnw', 'Slemr', 'Jakub']
Index: 24, Player ID

KeyboardInterrupt: 

In [118]:
print(player_list)
import csv
with open('PlayerList.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a row in the CSV file
    writer.writerow(player_list)
#player_list_df = pd.DataFrame(player_list, columns=["ID","LastName","FirstName"])
#player_list_df.to_csv("UniquePlayerList.csv",index=False)

[['f917lein', 'Swarowski', 'Robert'], ['yqvwjvgy', 'Pisicchio', 'Gabriele'], ['jkdsvj4j', 'Ishida', 'Itaru'], ['fodt8dk9', 'Grubaugh', 'Ed'], ['k9jc66ma', 'Sikorsky', 'Claudio'], ['tga82sa2', 'Franzen', 'Johan'], ['6hrzkc4m', 'Turian', 'Michael'], ['x1lebg8w', 'Kibler', 'Brian'], ['l98twjlh', 'Nishioka', 'Andrew'], ['imygnksk', 'Phillipps', 'Eric'], ['i2kzizct', 'Matarese', 'Chris'], ['sannwwjw', 'Schraut', 'Andreas'], ['h6ke3hcc', 'Wolf', 'Andrew'], ['r99or99j', 'Ladra', 'Lukas'], ['lewcamij', 'Ogilbee', 'Brian'], ['d7k7077v', 'Glodt', 'Romain'], ['me4nlw5o', 'Guevin', 'Thomas'], ['pzh1wbfb', 'Moss-Allison', 'Nicolas'], ['6mia5w8m', 'Konstanczer', 'Andre'], ['897497lk', 'Chinnock', 'John'], ['gpnoyxve', 'Eskeland', 'Sigurd'], ['lvzgd7gn', 'Eikefet', 'Kim'], ['z2315xnw', 'Slemr', 'Jakub'], ['tzu75jdp', 'Selden', 'Brian'], ['pzh3fcim', 'Levy', 'Raphael'], ['akq5t76h', 'Rubin', 'Benjamin'], ['3eudavoh', 'Johns', 'Scott'], ['nwnjg4a4', 'Buehler', 'Randy'], ['er8rx6et', 'Rodrigues Leone', 